In [ ]:
import os

os.getcwd()

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="eGjHxmLCAIMYJ89pbtFB")
project = rf.workspace("eeetan").project("doors-oyetk-kfpop-gv5s4")
version = project.version(1)
dataset = version.download("yolov11")

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO('yolo11n.pt')

In [3]:
results = model.train(data="C://Users/Eeeta/Geesehacks/hyacinthe/model_training/doors-1/data.yaml", epochs=20, imgsz=640,batch=8)

NameError: name 'model' is not defined

In [3]:
trained = YOLO('sign.pt')

In [7]:
import os
from ultralytics import YOLO
import cv2
from doctr.models import ocr_predictor

# Initialize the OCR model
model = ocr_predictor(pretrained=True)

# Open the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

ids = set()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture image.")
        break

    # Run YOLO inference on the frame
    results = trained.track(frame, persist=True, verbose=False, tracker='bytetrack.yaml', show=True, conf=0.65)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()  # YOLOv8 has built-in plotting for results

    # Process each detected object
    for result in results[0].boxes:
        # Extract bounding box, confidence, and class
        box = result.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2]
        conf = result.conf.cpu().item()     # Confidence score
        cls = int(result.cls.cpu().item())  # Class index
        if(result.id is not None):
            if(result.id not in ids):
                id = int(result.id.cpu().item())

                # Crop the frame around the detected object
                x1, y1, x2, y2 = map(int, box)
                cropped_object = frame[y1:y2, x1:x2]

                # Convert the cropped image to RGB
                cropped_object_rgb = cv2.cvtColor(cropped_object, cv2.COLOR_BGR2RGB)

                # Use doctr to predict text
                response = model([cropped_object_rgb])

                print(response)
                
                text = ""

                for page in response.pages:
                    for block in page.blocks:
                        for line in block.lines:
                            for word in line.words:
                                if(word.confidence > 0.5):
                                    text += word.value
                            text += "\n"

                print(text)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close the window
cap.release()
cv2.destroyAllWindows()